# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [82]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Ceck the dataframes for data wrangling

In [83]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [84]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [85]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


### Start with portfolio dataframe

In [86]:
# Split channels and offer_type into dummy variables
dummies = pd.get_dummies(portfolio['offer_type'])
dummies

,bogo,discount,informational
0,1,0,0
1,1,0,0
2,0,0,1
3,1,0,0
4,0,1,0
5,0,1,0
6,0,1,0
7,0,0,1
8,1,0,0
9,0,1,0


In [87]:
portfolio = pd.concat([portfolio,dummies],axis=1)
portfolio.drop('offer_type',axis=1)

,channels,difficulty,duration,id,reward,bogo,discount,informational
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,10,1,0,0
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10,1,0,0
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,0,0,0,1
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5,1,0,0
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,0,1,0
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,3,0,1,0
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,2,0,1,0
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,0,0,0,1
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,5,1,0,0
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,2,0,1,0


In [88]:
for i in range(len(portfolio.channels)):
    for j in portfolio.loc[i,"channels"]:
        portfolio.loc[i,j]=1 
portfolio.head()
portfolio.email=portfolio.email.fillna(0)
portfolio.mobile=portfolio.mobile.fillna(0)
portfolio.social=portfolio.social.fillna(0)
portfolio.web=portfolio.web.fillna(0)
portfolio.head()
portfolio.drop('channels',axis=1)

,difficulty,duration,id,offer_type,reward,bogo,discount,informational,email,mobile,social,web
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,1,0,0,1.0,1.0,1.0,0.0
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,0,0,1.0,1.0,1.0,1.0
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,0,0,1,1.0,1.0,0.0,1.0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,0,0,1.0,1.0,0.0,1.0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,0,1,0,1.0,0.0,0.0,1.0
5,7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3,0,1,0,1.0,1.0,1.0,1.0
6,10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2,0,1,0,1.0,1.0,1.0,1.0
7,0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0,0,0,1,1.0,1.0,1.0,0.0
8,5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5,1,0,0,1.0,1.0,1.0,1.0
9,10,7,2906b810c7d4411798c6938adc9daaa5,discount,2,0,1,0,1.0,1.0,0.0,1.0


### Continue with profile dataframe 

In [89]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [90]:
c = 0
for i in profile.age:
    if len(str(i))>2:
        profile.loc[c,'age'] = str(i)[1:]
    c+=1
profile.head()

,age,became_member_on,gender,id,income
0,18,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,18,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,18,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [91]:
profile.became_member_on=profile.became_member_on.astype(str)
profile.loc[:,'became_member_on'] = profile.loc[:,'became_member_on'].str[:4]+'.'+profile.loc[:,'became_member_on'].str[4:6]+'.'+profile.loc[:,'became_member_on'].str[6:]
profile.head()

,age,became_member_on,gender,id,income
0,18,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,2017.07.15,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,18,2018.07.12,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,2017.05.09,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,18,2017.08.04,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [93]:
profile.income = profile.income.fillna(0)
profile.head()

,age,became_member_on,gender,id,income
0,18,2017.02.12,None,68be06ca386d4c31939f3a4f0e3dd783,0.0
1,55,2017.07.15,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,18,2018.07.12,None,38fe809add3b4fcf9315a9694bb96ff5,0.0
3,75,2017.05.09,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,18,2017.08.04,None,a03223e636434f42ac4c3df47e8bac43,0.0


### Finish with transcript dataframe

In [99]:
dummies = pd.get_dummies(transcript.event)
dummies
transcript = pd.concat([transcript,dummies],axis=1)
transcript = transcript.drop('event',axis=1)
transcript.head()

,person,time,value,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0


In [104]:
transcript = transcript.rename(columns={"person":"id"})
transcript.head()

,id,time,value,offer completed,offer received,offer viewed,transaction
0,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,1,0,0
2,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,1,0,0
3,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0,1,0,0
4,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0,1,0,0


In [111]:
transcript[100000:]

,id,time,value,offer completed,offer received,offer viewed,transaction
100000,75346ec64fd44dd196e782cbeab000ef,270,{'amount': 10.65},0,0,0,1
100001,add8153f744b485b96901eb9cd066f05,270,{'amount': 11.21},0,0,0,1
100002,ab2dfcc6a76844a9a5a37d9d93a74ff7,270,{'amount': 0.05},0,0,0,1
100003,37fe1a9de1c1480e8557d097835d7c0f,270,{'amount': 8.36},0,0,0,1
100004,acc974aac2684d0c9d7dea71c4fb412f,270,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0,0,1,0
100005,acc974aac2684d0c9d7dea71c4fb412f,270,{'amount': 33.96},0,0,0,1
100006,acc974aac2684d0c9d7dea71c4fb412f,270,{'offer_id': 'ae264e3637204a6fb9bb56bc8210ddfd...,1,0,0,0
100007,b72b74ce11254d309e8c93a47ed51032,270,{'amount': 25.42},0,0,0,1
100008,7d650753a27846f5a7d9b1f5eb748291,270,{'amount': 27.86},0,0,0,1
100009,7d650753a27846f5a7d9b1f5eb748291,270,{'offer_id': '0b1e1539f2cc45b7b9fa7c272da2e1d7...,1,0,0,0


### Merge everything to start the analysis

### Resources
- https://stackoverflow.com/questions/5254445/add-string-in-a-certain-position-in-python  
- 